In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tinashri/brain-tumor-dataset-includes-the-mask-and-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brain-tumor-dataset-includes-the-mask-and-images


In [2]:
import os

dataset_path = "/kaggle/input/brain-tumor-dataset-includes-the-mask-and-images/data/data"
print("Images folder exists:", os.path.exists(os.path.join(dataset_path, "images")))
print("Masks folder exists:", os.path.exists(os.path.join(dataset_path, "masks")))


Images folder exists: True
Masks folder exists: True


In [3]:
from glob import glob

images = glob(f"{dataset_path}/images/*")
masks = glob(f"{dataset_path}/masks/*")

print(f"Found {len(images)} images and {len(masks)} masks")


Found 3064 images and 3064 masks


In [4]:
import os
import numpy as np
import torch
import cv2
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms

# Set image dimensions
H, W = 256, 256

# Define dataset class
class BrainTumorDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image and mask
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize to fixed dimensions
        image = cv2.resize(image, (H, W))
        mask = cv2.resize(mask, (H, W))

        # Normalize image (scale pixel values to [0,1])
        image = image.astype(np.float32) / 255.0
        mask = mask.astype(np.float32) / 255.0

        # Convert to tensors
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # (C, H, W)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # (1, H, W)

        if self.transform:
            image = self.transform(image)

        return image, mask

# Load dataset paths
def load_dataset(path, split=0.1):
    image_dir = os.path.join(path, "images")
    mask_dir = os.path.join(path, "masks")

    image_paths = sorted(glob(f"{image_dir}/*"))
    mask_paths = sorted(glob(f"{mask_dir}/*"))

    if len(image_paths) == 0 or len(mask_paths) == 0:
        raise ValueError(f"No images or masks found in {path}. Check dataset structure.")

    # Split data
    total_size = len(image_paths)
    test_size = int(total_size * split)
    valid_size = int(total_size * split)
    train_size = total_size - (test_size + valid_size)

    dataset = BrainTumorDataset(image_paths, mask_paths)
    train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

    return train_dataset, valid_dataset, test_dataset

# Create DataLoader
def create_dataloaders(dataset_path, batch_size=16):
    train_dataset, valid_dataset, test_dataset = load_dataset(dataset_path)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

# Main script
if __name__ == "__main__":
    dataset_path = "/kaggle/input/brain-tumor-dataset-includes-the-mask-and-images/data/data"

    # Create DataLoaders
    train_loader, valid_loader, test_loader = create_dataloaders(dataset_path, batch_size=16)

    # Print dataset sizes
    print(f"Train: {len(train_loader.dataset)} images")
    print(f"Validation: {len(valid_loader.dataset)} images")
    print(f"Test: {len(test_loader.dataset)} images")

    # Get a batch of images and masks
    sample_images, sample_masks = next(iter(train_loader))
    print(f"Sample batch shape - Images: {sample_images.shape}, Masks: {sample_masks.shape}")


Train: 2452 images
Validation: 306 images
Test: 306 images
Sample batch shape - Images: torch.Size([16, 3, 256, 256]), Masks: torch.Size([16, 1, 256, 256])


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define U-Net Model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder (Downsampling)
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder (Upsampling)
        self.upconv4 = self.upconv(1024, 512)
        self.dec4 = self.conv_block(1024, 512)

        self.upconv3 = self.upconv(512, 256)
        self.dec3 = self.conv_block(512, 256)

        self.upconv2 = self.upconv(256, 128)
        self.dec2 = self.conv_block(256, 128)

        self.upconv1 = self.upconv(128, 64)
        self.dec1 = self.conv_block(128, 64)

        # Final Convolution Layer
        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def upconv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(nn.MaxPool2d(2)(enc1))
        enc3 = self.enc3(nn.MaxPool2d(2)(enc2))
        enc4 = self.enc4(nn.MaxPool2d(2)(enc3))

        # Bottleneck
        bottleneck = self.bottleneck(nn.MaxPool2d(2)(enc4))

        # Decoder
        up4 = self.upconv4(bottleneck)
        up4 = torch.cat([up4, enc4], dim=1)
        dec4 = self.dec4(up4)

        up3 = self.upconv3(dec4)
        up3 = torch.cat([up3, enc3], dim=1)
        dec3 = self.dec3(up3)

        up2 = self.upconv2(dec3)
        up2 = torch.cat([up2, enc2], dim=1)
        dec2 = self.dec2(up2)

        up1 = self.upconv1(dec2)
        up1 = torch.cat([up1, enc1], dim=1)
        dec1 = self.dec1(up1)

        return torch.sigmoid(self.final_conv(dec1))  # Sigmoid for binary segmentation

# Instantiate Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)


In [6]:
# Dice Coefficient Metric
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    intersection = (y_true * y_pred).sum()
    return (2. * intersection + smooth) / (y_true.sum() + y_pred.sum() + smooth)

# Dice Loss
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, y_pred, y_true):
        return 1 - dice_coef(y_true, y_pred)

# Define Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
dice_loss = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [7]:
from tqdm import tqdm

# Train function
def train(model, train_loader, optimizer, criterion, dice_loss, device):
    model.train()
    total_loss, total_dice = 0, 0

    for images, masks in tqdm(train_loader):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        bce = criterion(outputs, masks)
        dice = dice_loss(outputs, masks)
        loss = bce + dice

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_dice += dice_coef(masks, outputs).item()

    return total_loss / len(train_loader), total_dice / len(train_loader)

# Validation function
def validate(model, valid_loader, criterion, dice_loss, device):
    model.eval()
    total_loss, total_dice = 0, 0

    with torch.no_grad():
        for images, masks in tqdm(valid_loader):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            bce = criterion(outputs, masks)
            dice = dice_loss(outputs, masks)
            loss = bce + dice

            total_loss += loss.item()
            total_dice += dice_coef(masks, outputs).item()

    return total_loss / len(valid_loader), total_dice / len(valid_loader)


In [8]:
# Load Data
train_loader, valid_loader, test_loader = create_dataloaders("/kaggle/input/brain-tumor-dataset-includes-the-mask-and-images/data/data", batch_size=16)

# Training Loop
num_epochs = 20
best_valid_loss = float("inf")

for epoch in range(num_epochs):
    train_loss, train_dice = train(model, train_loader, optimizer, criterion, dice_loss, device)
    valid_loss, valid_dice = validate(model, valid_loader, criterion, dice_loss, device)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Train Dice: {train_dice:.4f} - Valid Loss: {valid_loss:.4f} - Valid Dice: {valid_dice:.4f}")

    # Save Best Model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "best_unet.pth")
        print("Model Saved!")


100%|██████████| 20/20 [00:13<00:00,  1.52it/s]


Epoch 1/20 - Train Loss: 1.7322 - Train Dice: 0.0931 - Valid Loss: 1.6815 - Valid Dice: 0.1192
Model Saved!


100%|██████████| 20/20 [00:07<00:00,  2.67it/s]


Epoch 2/20 - Train Loss: 1.6434 - Train Dice: 0.1358 - Valid Loss: 1.6134 - Valid Dice: 0.1547
Model Saved!


100%|██████████| 20/20 [00:07<00:00,  2.65it/s]


...

Epoch 18/20 - Train Loss: 0.8358 - Train Dice: 0.8534 - Valid Loss: 0.9113 - Valid Dice: 0.7784


100%|██████████| 20/20 [00:07<00:00,  2.52it/s]


Epoch 19/20 - Train Loss: 0.8257 - Train Dice: 0.8632 - Valid Loss: 0.8684 - Valid Dice: 0.8210
Model Saved!


100%|██████████| 20/20 [00:07<00:00,  2.59it/s]

Epoch 20/20 - Train Loss: 0.8203 - Train Dice: 0.8686 - Valid Loss: 0.9151 - Valid Dice: 0.7747


In [9]:
# Load Best Model
model.load_state_dict(torch.load("best_unet.pth"))

# Evaluate on Test Data
test_loss, test_dice = validate(model, test_loader, criterion, dice_loss, device)
print(f"Test Loss: {test_loss:.4f} - Test Dice Score: {test_dice:.4f}")

<ipython-input-9-4a856688fb6c>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_unet.pth"))
100%|██████████| 20/20 [00:14<00:00,  1.42

Test Loss: 0.9056 - Test Dice Score: 0.7844
